In [193]:
import pandas as pd
import re

In [194]:
df = pd.read_csv('../ressources/babyfoot_dataset.csv')

/var/folders/l2/4bhymbh53d3_vrys9v8mb6hw0000gn/T/ipykernel_67156/4222138916.py:1: DtypeWarning: Columns (10,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../ressources/babyfoot_dataset.csv')


In [195]:
# Suppression des colonnes inutiles
df.drop([
        'table_condition',
        'ball_type', 
        'music_playing', 
        'referee', 
        'attendance_count',
        'season', 
        'recorded_by', 
        'rating_raw', 
        'player_goals',
        'player_own_goals', 
        'player_age',
        'player_assists', 
        'player_saves', 
        'possession_time',
        'mood', 
        'player_comment', 
        'is_substitute', 
        'ping_ms',
        'notes', 
        'duplicate_flag',
        'misc', 
        'created_at'],
    axis=1, inplace=True)

In [196]:
# Nettoyage et formatage de la colonne date 

# Suppression des caractères parasites
def clean_date(date_str):
    if pd.isna(date_str):
        return date_str
    date_str = str(date_str)
    date_str = re.sub(r'(\d+)(st|nd|rd|th)', r'\1', date_str)
    date_str = date_str.replace('.', '')
    return date_str


# Conversion des différents formats de date
def convert_date(date_str):
    if pd.isna(date_str):
        return pd.NaT
    date_str = str(date_str).strip()
    
    # Liste des formats possibles
    formats = [
        '%b %d %Y',        # Feb 06 2023
        '%d-%m-%Y',        # 24-03-2023
        '%Y-%m-%d',        # 2025-01-13
        '%d %b %y',        # 30 Sep 23
        '%Y/%m/%d',        # 2023/07/04
        '%d/%m/%y',        # 24/06/23
        '%m/%d/%Y',        # 02/18/2024
        '%d %b %Y',        # 20 Mar 2023 (avec année complète)
    ]
    
    # Essayer chaque format
    for format in formats:
        try:
            return pd.to_datetime(date_str, format=format)
        except:
            continue
    
    # Si aucun format ne marche, retourner NaT
    return pd.NaT


df['game_date'] = df['game_date'].apply(clean_date)
df['game_date'] = df['game_date'].apply(convert_date)

In [197]:
# Nettoyage de la colonne game_duration


def convert_in_second(duration):
    """Convertit différents formats de durée en secondes (int)"""
    
    if pd.isna(duration):
        return None
    
    duration_str = str(duration).strip()
    
    # Format 1: "6min" ou "6 min"
    if 'min' in duration_str.lower():
        minutes = int(re.search(r'(\d+)', duration_str).group(1))
        return minutes * 60
    
    # Format 2: "00:05:55" (heures:minutes:secondes)
    elif ':' in duration_str:
        parties = duration_str.split(':')
        if len(parties) == 3:  # HH:MM:SS
            heures, minutes, secondes = map(int, parties)
            return heures * 3600 + minutes * 60 + secondes
        elif len(parties) == 2:  # MM:SS
            minutes, secondes = map(int, parties)
            return minutes * 60 + secondes
    
    # Format 3: "12.45" (minutes.secondes)
    elif '.' in duration_str:
        parties = duration_str.split('.')
        if parties[0].isdigit() and parties[1].isdigit():
            minutes = int(parties[0])
            secondes = int(parties[1])
            return minutes * 60 + secondes

    # Si c'est juste un nombre entier
    else:
        return int(float(duration_str))
    
    return None

df['game_duration'] = df['game_duration'].apply(convert_in_second)
df['game_duration'] = df['game_duration'].fillna(0).astype(int)


In [198]:
# Nettoyage des colonnes game_id, game_id et player_id pour ne garder qu'un format int
# pour les rendre compatibles avec les futures données

df['game_id'] = df['game_id'].str.extract('(\d+)').astype(int)
df['table_id'] = df['table_id'].str.extract('(\d+)').astype(int)
df['player_id'] = df['player_id'].str.extract('(\d+)').astype(int)


<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:6: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:6: SyntaxWarning: invalid escape sequence '\d'
/var/folders/l2/4bhymbh53d3_vrys9v8mb6hw0000gn/T/ipykernel_67156/2683099155.py:4: SyntaxWarning: invalid escape sequence '\d'
  df['game_id'] = df['game_id'].str.extract('(\d+)').astype(int)
/var/folders/l2/4bhymbh53d3_vrys9v8mb6hw0000gn/T/ipykernel_67156/2683099155.py:5: SyntaxWarning: invalid escape sequence '\d'
  df['table_id'] = df['table_id'].str.extract('(\d+)').astype(int)
/var/folders/l2/4bhymbh53d3_vrys9v8mb6hw0000gn/T/ipykernel_67156/2683099155.py:6: SyntaxWarning: invalid escape sequence '\d'
  df['player_id'] = df['player_id'].str.extract('(\d+)').astype(int)


In [199]:
# Nettoyage des colonnes de score

# 1. Sélectionner les lignes concernées
lignes_a_traiter = df['final_score_blue'].isna()

# 2. Séparer le score en deux colonnes
scores_separes = df.loc[lignes_a_traiter, 'final_score_red'].str.split(r'\s*-\s*', expand=True, regex=True)

# 3. Assigner aux bonnes colonnes (et convertir en int)
df.loc[lignes_a_traiter, 'final_score_blue'] = scores_separes[0]
df.loc[lignes_a_traiter, 'final_score_red'] = scores_separes[1]
df['final_score_blue'] = pd.to_numeric(df['final_score_blue'], errors='coerce').astype('Int64')
df['final_score_red'] = pd.to_numeric(df['final_score_red'], errors='coerce').astype('Int64')

# 4. Supprimer les lignes avec des valeurs nulles
df = df.dropna(subset=['final_score_blue', 'final_score_red'])

In [200]:
# Nettoyage de la colonne winner

# 1. Mettre en MAJUSCULES et nettoyer les espaces
df['winner'] = df['winner'].str.strip().str.upper()

# 2. Créer un mapping pour uniformiser
mapping = {
    'BLUE': 'BLUE',
    'BLEU': 'BLUE',
    'B': 'BLUE',
    'RED': 'RED',
    'R': 'RED',
    'TIE': 'TIE',
    'DRAW': 'TIE',
    'ÉGALITÉ': 'TIE'
}

# 3. Appliquer le mapping
df['winner'] = df['winner'].map(mapping)

# 4. Supprimer les valeurs invalides (pas 'BLUE', 'RED', 'TIE', ou NaN)

valeurs_valides = ['BLUE', 'RED', 'TIE']

lignes_invalides = ~(df['winner'].isin(valeurs_valides) | df['winner'].isna())

if lignes_invalides.sum() > 0:
    df = df[~lignes_invalides].copy()


# 5. Attribuer un gagnant si le winner est NaN

def determiner_gagnant(row):
    """
    Compare les scores et retourne le gagnant
    """
    blue = row['final_score_blue']
    red = row['final_score_red']
    
    # Si un des scores est NaN, on ne peut pas déterminer
    if pd.isna(blue) or pd.isna(red):
        return None
    
    if blue > red:
        return 'BLUE'
    elif red > blue:
        return 'RED'
    else:
        return 'TIE'


# Identifier les lignes avec NaN dans winner
lignes_nan = df['winner'].isna()
df.loc[lignes_nan, 'winner'] = df.loc[lignes_nan].apply(determiner_gagnant, axis=1)


# 6. Vérifier la cohérence des winner déjà existants

def verifier_coherence(df):
    """
    Vérifie que le winner correspond aux scores
    """
    def est_coherent(row):
        if pd.isna(row['winner']) or pd.isna(row['final_score_blue']) or pd.isna(row['final_score_red']):
            return True  # On ne peut pas vérifier
        
        blue = row['final_score_blue']
        red = row['final_score_red']
        winner = row['winner']
        
        if blue > red and winner == 'BLUE':
            return True
        elif red > blue and winner == 'RED':
            return True
        elif blue == red and winner == 'TIE':
            return True
        else:
            return False
    
    coherence = df.apply(est_coherent, axis=1)
    incoherentes = ~coherence
    
    print(f"\n=== VÉRIFICATION COHÉRENCE ===")
    print(f"Lignes cohérentes: {coherence.sum()}")
    print(f"Lignes incohérentes: {incoherentes.sum()}")
    
    if incoherentes.sum() > 0:
        print("\n⚠️ Lignes incohérentes:")
        print(df.loc[incoherentes, ['game_id', 'final_score_blue', 'final_score_red', 'winner']])
    
    return df[coherence].copy()

df = verifier_coherence(df)





=== VÉRIFICATION COHÉRENCE ===
Lignes cohérentes: 98878
Lignes incohérentes: 1320

⚠️ Lignes incohérentes:
        game_id  final_score_blue  final_score_red winner
28        12583                 0                1   BLUE
31        10585                 0                1   BLUE
76         1519                 4                2    RED
80        10274                 4                0    RED
114        1922                 7                5    RED
...         ...               ...              ...    ...
99545      7518                 1                9   BLUE
99582      8930                10                8    RED
99618       763                 0               10   BLUE
99639     12520                 5                5   BLUE
100182    18022                 2                7   BLUE

[1320 rows x 4 columns]


In [201]:
# Nettoyage de la colonne player_role
# On dénifit 3 valeurs possibles ATK, DEF et FULL pour les 1v1

df['player_role'] = df['player_role'].str.strip().str.upper()

mapping = {
    'ATTACK': 'ATK',
    'ATTCK': 'ATK', 
    'ATK': 'ATK',
    'ATACK': 'ATK',

    'DEFENSE': 'DEF',
    'DEFENCE': 'DEF', 
    'DEF': 'DEF',
}

df['player_role'] = df['player_role'].map(mapping)

# Suppression des lignes où le rôle n'est pas défini
df = df[df['player_role'].notna()]


In [202]:
# Nettoyage de team_color 

df['team_color'] = df['team_color'].str.strip().str.upper()

mapping = {
    'RED': 'RED',
    'R': 'RED',
    'ROUGE': 'RED',
    '🔴': 'RED',
    'RED�': 'RED',  # Caractère corrompu
    
    'BLUE': 'BLUE',
    'BLEU': 'BLUE',
    'B': 'BLUE',
    '🔵': 'BLUE',
    'BLUE�': 'BLUE'  # Caractère corrompu
}

df['team_color'] = df['team_color'].map(mapping)

# Suppression des valeurs NaN
df = df.dropna(subset=['team_color'])

In [203]:
# Nettoyage et mise en forme des noms de joueurs

df[['name', 'surname']] = df['player_canonical_name'].str.split(' ', expand=True)
df['surname'] = df['surname'].str.upper()
df['name'] = df['name'].str.capitalize()
df = df.drop('player_canonical_name', axis=1)

In [204]:
nouveau_noms = {
    'game_date': 'date',
    'game_duration': 'duration',
    'final_score_red': 'red_goal',
    'final_score_blue': 'blue_goal',
    'team_color': 'team',
    'player_role': 'role',
    'player_name': 'pseudo',
}

df = df.rename(columns=nouveau_noms)

In [ ]:
# Création des différents csv, un par table + renommage des colonnes

df_user = df[
    [
        'player_id', 
        'name', 
        'surname']
    ].rename(columns={
        'player_id': 'id'
})

df_user_game = df[
    [
        'player_id', 
        'game_id', 
        'role',
        'team']
    ]

df_game = df[
    [
        'game_id', 
        'date', 
        'table_id',
        'duration',
        'red_goal',
        'blue_goal',
        'winner',
        ]
    ].rename(columns={
        'game_id': 'id'
})

df_table = df[
    [
        'table_id', 
        'location']
    ].rename(columns={
        'table_id': 'id'
})

# Chaque table devrait en théorie contenir des lignes uniques (id)
df_user.drop_duplicates(subset=['id'], keep='first')
df_user_game.drop_duplicates(subset=[''])

# Exporter directement
df_user.to_csv('user.csv', index=False)
df_user_game.to_csv('user_game.csv', index=False)
df_game.to_csv('game.csv', index=False)
df_table.to_csv('table.csv', index=False)

In [209]:
# Export du dataset propre

df.to_csv('dataset_clean.csv', index=False)

In [211]:
df_user_game.duplicated().sum()

196